[periodo-reconciler/API.md at master · periodo/periodo-reconciler](https://github.com/periodo/periodo-reconciler/blob/master/API.md)

In [1]:
import requests
import json

In [2]:
from periodo_reconciler import (
    RProperty,
    RQuery,
    PeriodoReconciler
)

Instantiate `PeriodoReconciler` for the reconciler run on the default local port

In [3]:
p_recon = PeriodoReconciler(host='localhost:8142')
p_recon

PeriodoReconciler(host="localhost:8142", protocol="http")

Call the [describe method of the reconciler](https://github.com/periodo/periodo-reconciler/blob/master/API.md#describe-the-reconciliation-service)

In [4]:
set(p_recon.describe().keys())

{'defaultTypes',
 'identifierSpace',
 'name',
 'preview',
 'schemaSpace',
 'suggest',
 'view'}

# Reconcile period names 


Let's cast the example from [periodo-reconciler/API.md at master · periodo/periodo-reconciler](https://github.com/periodo/periodo-reconciler/blob/master/API.md#reconcile-period-names-using-get), which has the following  a JSON object specifying three queries: 


```
{
  "basic-query": {
    "query": "北宋"
  },
  "limited-query": {
    "query": "bronze age",
    "limit": 1
  },
  "additional-properties-query": {
    "query": "Ранньоримський",
    "properties": [
      {
        "p": "location",
        "v": "Ukraine"
      },
      {
        "p": "start",
        "v": "200"
      },
      {
        "p": "stop",
        "v": "600"
      }
    ]
  }
}
```

In [5]:
# q is the JSON object specifying three queries

q = """
{
  "basic-query": {
    "query": "北宋"
  },
  "limited-query": {
    "query": "bronze age",
    "limit": 1
  },
  "additional-properties-query": {
    "query": "Ранньоримський",
    "properties": [
      {
        "p": "location",
        "v": "Ukraine"
      },
      {
        "p": "start",
        "v": "200"
      },
      {
        "p": "stop",
        "v": "600"
      }
    ]
  }
}

""".strip()

json.loads(q)

{'basic-query': {'query': '北宋'},
 'limited-query': {'query': 'bronze age', 'limit': 1},
 'additional-properties-query': {'query': 'Ранньоримський',
  'properties': [{'p': 'location', 'v': 'Ukraine'},
   {'p': 'start', 'v': '200'},
   {'p': 'stop', 'v': '600'}]}}

Call the Open Refine reconciliation service directly using `requests`

In [6]:
r = requests.get('http://localhost:8142', params={'queries':q})
r.json()

{'basic-query': {'result': []},
 'limited-query': {'result': []},
 'additional-properties-query': {'result': []}}

Now, rewrite this payload as a list of `RQuery` objects, some of which themselves contain some `RProperty` instances.

In [7]:
queries = [
    RQuery("北宋", label="basic-query"),
    RQuery("bronze age", label="limited-query", limit=1),
    RQuery("Ранньоримський", label="additional-properties-query", properties=[
        RProperty('location', 'Ukraine'),
        RProperty('start', 200),
        RProperty('end', 600)
    ])
]

queries

[RQuery("\u5317\u5b8b", label="basic-query"),
 RQuery("bronze age", label="limited-query", limit=1),
 RQuery("\u0420\u0430\u043d\u043d\u044c\u043e\u0440\u0438\u043c\u0441\u044c\u043a\u0438\u0439", label="additional-properties-query", properties=[RProperty("location", "Ukraine"),
 RProperty("start", 200),
 RProperty("end", 600)])]

In [8]:
# you can use post or get and get the same results in doing the reconciliation
r = p_recon.reconcile(queries, method='post')
r

{'basic-query': {'result': []},
 'limited-query': {'result': []},
 'additional-properties-query': {'result': []}}

Demonstrate how to [Suggest property names](https://github.com/periodo/periodo-reconciler/blob/master/API.md#suggest-property-names)

In [9]:
result = p_recon.suggest_properties()
result

[{'id': 'location', 'name': 'Spatial coverage'},
 {'id': 'start', 'name': 'Year or start year'},
 {'id': 'stop', 'name': 'End year'}]

Exercise the [suggest entities](https://github.com/periodo/periodo-reconciler/blob/master/API.md#suggest-period-definitions) functionality

In [15]:
# http://localhost:8142/suggest/entities?prefix=bashkekohore

p_recon.suggest_entities('bashkekohore')

[]

In [16]:
# suggest_entities doesn't seem to be working anymore

p_recon.suggest_entities('Iron')

[]

[Preview period definition details](https://github.com/periodo/periodo-reconciler/blob/master/API.md#preview-period-definition-details)

In [11]:
p_recon.preview_period('http://n2t.net/ark:/99152/p0fp7wvjvn8')

HTTPError: 404 Client Error: Not Found for url: http://localhost:8142/preview?id=http%3A%2F%2Fn2t.net%2Fark%3A%2F99152%2Fp0fp7wvjvn8

In [19]:
r = p_recon.preview_period('http://n2t.net/ark:/99152/p0fp7wvjvn8', flyout=True)

HTTPError: 404 Client Error: Not Found for url: http://localhost:8142/preview?id=http%3A%2F%2Fn2t.net%2Fark%3A%2F99152%2Fp0fp7wvjvn8&flyout=True

In [ ]:
RProperty('location', 'Ukraine')

In [ ]:
# a label is automatically created for RQuery if non is provided

RQuery("bronze age")

In [ ]:
RQuery("bronze age", label="limited-query", limit=1)

In [ ]:
RQuery("bronze age", label="limited-query")

In [ ]:
RQuery("Ранньоримський", label="additional-properties-query", properties=[
        RProperty('location', 'Ukraine'),
        RProperty('start', 200),
        RProperty('end', 600)
    ])

In [ ]:
q = RQuery("Ранньоримський", label="additional-properties-query", properties=[
        RProperty('location', 'Ukraine'),
        RProperty('start', 200),
        RProperty('end', 600)
    ])

In [ ]:
repr(q)

# PKAP: comparing "Late Roman" vs "Roman, Late"

In [17]:
queries = [

    RQuery("Late Roman", label="PKAP Late Roman", properties=[
        RProperty('location', 'Cyprus'),
        RProperty('start', 300),
        RProperty('end', 749)
    ]), 
    RQuery("Roman, Late", label="PKAP Roman, Late",  properties=[
        RProperty('location', 'Cyprus'),
        RProperty('start', 300),
        RProperty('end', 749)
    ]),    

]

queries

[RQuery("Late Roman", label="PKAP Late Roman", properties=[RProperty("location", "Cyprus"),
 RProperty("start", 300),
 RProperty("end", 749)]),
 RQuery("Roman, Late", label="PKAP Roman, Late", properties=[RProperty("location", "Cyprus"),
 RProperty("start", 300),
 RProperty("end", 749)])]

In [18]:
r = p_recon.reconcile(queries, method='post')
r

{'PKAP Late Roman': {'result': []}, 'PKAP Roman, Late': {'result': []}}

Conclusion: "Roman, Late" brings up no results whereas "Late Roman" brings up many results.